In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn import cluster, feature_extraction, model_selection

In [24]:
dat = open(r'checkins.dat', 'r')

In [25]:
lines = dat.readlines()
lines.pop(1)

'---------+---------+----------+-------------------+-------------------+---------------------\n'

In [33]:
with open (r'checking.csv', 'w') as csv:
    for line in lines:
        line = line.replace('|', ';')
        line = line.replace(' ', '')
        csv.write(line)
csv.close

<function TextIOWrapper.close()>

In [34]:
data = pd.read_csv(r'checking.csv', sep = ';', engine = 'python')

In [35]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-2117:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-2117:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-2117:42:58


In [39]:
data.query('latitude != "NaN" or longitude != "NaN"', inplace = True)

In [43]:
data.dropna(subset = ['latitude', 'longitude'], inplace = True)

In [45]:
data_new = data[:10**5]

In [46]:
data_new

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-2117:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-2117:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-2117:38:18
11,984483,1030290.0,955969.0,32.221743,-110.926479,2012-04-2117:58:54
12,984685,304253.0,23558.0,40.650000,-73.950000,2012-04-2118:19:34
13,984470,720850.0,749715.0,33.448377,-112.074037,2012-04-2117:02:47
15,984610,1639666.0,442605.0,33.414768,-111.909309,2012-04-2118:04:58
18,984653,1647192.0,23558.0,42.358431,-71.059773,2012-04-2118:23:22


In [47]:
cluster = cluster.MeanShift(bandwidth = 0.1)

In [62]:
X = np.transpose(np.array([data_new['latitude'], data_new['longitude']]))
X

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ...,
       [  29.7628844,  -95.3830615],
       [  32.802955 ,  -96.769923 ],
       [  37.7749295, -122.4194155]])

In [63]:
cluster.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [107]:
counts = np.unique(cluster.labels_, return_counts = True)
counts

(array([   0,    1,    2, ..., 3228, 3229, 3230], dtype=int64),
 array([12506,  4692,  3994, ...,     1,     1,     1], dtype=int64))

In [112]:
special_arr = []
for i in range(0, len(counts[0])):
    if counts[1][i] > 15:
        special_arr.append(counts[0][i])

In [131]:
cluster.cluster_centers_[0]

array([ 40.7177164 , -73.99183542])

In [190]:
special_arr[:]

SyntaxError: invalid syntax (<ipython-input-190-015970797129>, line 1)

In [141]:
big_clusters = []
for item in special_arr:
    big_clusters.append(cluster.cluster_centers_[item])
big_clusters = np.array(big_clusters)

In [140]:
offices = [[33.751277, -118.188740],
                   [25.867736, -80.324116],
                   [51.503016, -0.075479],
                   [52.378894, 4.885084],
                   [39.366487, 117.036146],
                   [-33.868457, 151.205134]]
offices = np.array(offices)
offices[0]

array([  33.751277, -118.18874 ])

In [174]:
distances = np.zeros((big_clusters.shape[0], offices.shape[0]))
coordinates = np.zeros((big_clusters.shape[0], offices.shape[0], 2))

In [207]:
k = 0
kk = 10**10
i = 0
j = 0
for office in offices:
    for coords in big_clusters:
        distances[j,i] = np.linalg.norm(np.array(office) - np.array(coords))
        coordinates[j,i,:] = coords
        j += 1
        if k < kk:
            kk = k
            nearest = coords
    i += 1
    j = 0

In [208]:
coords

array([ 38.65877915, -76.8856871 ])

In [209]:
offices[0] - big_clusters[0]

array([ -6.9664394 , -44.19690458])

In [210]:
np.ndarray.argmin(distances)

2507

In [211]:
index = np.unravel_index(distances.argmin(), distances.shape)

In [212]:
coordinates[index[0], index[1], :]

array([-33.86063043, 151.20477593])

In [206]:
coordinates[417,5]

array([-33.86063043, 151.20477593])

In [191]:
distances

array([[ 44.74257092,  16.14371999,  74.69906582,  79.73425538,
        191.0327603 , 237.22725876],
       [  6.19395917,  32.5726786 , 113.37331719, 118.41008135,
        229.11470442, 271.67953165],
       [  6.29424146,  32.47447468, 113.2748328 , 118.31160891,
        229.01456477, 271.58164907],
       ...,
       [ 30.76605219,  17.72133043,  88.92117683,  93.94890117,
        205.49405482, 251.2583223 ],
       [ 41.37357142,  13.74564883,  78.07422497,  83.1108233 ,
        194.21830835, 239.80611351],
       [ 41.5935783 ,  13.24513413,  77.87671339,  82.91381403,
        193.92312447, 239.34415107]])